# Importing Essential Libraries
This cell imports the necessary libraries for the project, sets up the device (GPU/CPU) for computations, and checks for available hardware accelerators like CUDA or MPS.

- **numpy**: Used for numerical computations (arrays, mathematical operations).
- **torch**: A deep learning framework used for building and training machine learning models.
  - `torch.nn`: Submodule for defining neural networks.
  - `torch.optim`: Provides optimization algorithms for training.
  - `torch.nn.functional`: Contains functions for layers, activations, and other operations.
  - `torch.utils.data`: Classes for managing datasets and data loaders.
- **torcheval.metrics**: Provides evaluation metrics (e.g., Word Error Rate, Perplexity).
- **re**: For text processing and pattern matching using regular expressions.
- **tqdm**: Adds a progress bar for long-running loops.
- **datasets**: Used to load NLP datasets from the Hugging Face library.
- **gensim**: Useful for NLP tasks like training Word2Vec models.

Finally, the code determines if a GPU (or other accelerator) is available for computations and falls back to the CPU if none is found.

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torcheval.metrics import WordErrorRate, Perplexity
import re

from tqdm import tqdm

from datasets import load_dataset

import gensim

if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

print('Using device:', device)

/Users/guilome/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: mps


# Loading and Preprocessing the Dataset
This cell performs several steps to prepare the dataset for use in a natural language processing (NLP) task:

### Step 1: Load the Dataset
- The **Wikitext-2-raw-v1** dataset is loaded using Hugging Face's `datasets` library.
  - `train`, `validation`, and `test` splits are extracted as lists of text strings.

### Step 2: Tokenization with Regular Expressions
- A custom function `tokenize_with_re` is defined to tokenize the text.
  - It uses regular expressions to extract words containing only alphabetic characters (`\b[a-zA-Z]+\b`).
  - All words are converted to lowercase for consistency.
  - Sentences with only whitespace are ignored.

### Step 3: Apply Tokenization
- The `tokenize_with_re` function is applied to the train, validation, and test datasets.
- After tokenization:
  - Each tokenized dataset split is flattened into a single list of words.
  - A combined list `flat_total` is created by merging all splits.

### Step 4: Build the Vocabulary
- A sorted set of unique words (`words_available`) is created from the combined word list.
- The vocabulary size is determined as the length of this set.

### Step 5: Create Mapping Dictionaries
- `stoi` (String-to-Int): Maps each word to its unique index.
- `itos` (Int-to-String): Maps each index back to its corresponding word.


In [2]:
train_data = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1")

train = train_data['train']['text']
valid = train_data['validation']['text']
test = train_data['test']['text']


# Step 2: Tokenize the text using regular expressions
def tokenize_with_re(data):
    tokenized_sentences = [
        re.findall(r'\b[a-zA-Z]+\b', sentence.lower()) for sentence in data if sentence.strip()
    ]
    return tokenized_sentences

# Tokenize each dataset split
train_tokenized = tokenize_with_re(train)
valid_tokenized = tokenize_with_re(valid)
test_tokenized = tokenize_with_re(test)

flat_train = [item for sublist in train_tokenized for item in sublist]
flat_valid = [item for sublist in valid_tokenized for item in sublist]
flat_test = [item for sublist in test_tokenized for item in sublist]

flat_total = flat_train + flat_valid + flat_test
words_available = sorted(set(flat_total))
vocab_size = len(words_available)

stoi = {word: i for i, word in enumerate(words_available)}
itos = {i: word for i, word in enumerate(words_available)}

print(f"Total number of words: {len(flat_total)}, vocabulary size: {len(words_available)}")

Total number of words: 2053065, vocabulary size: 66929


# Function to Create Sequences for Training
This function generates input sequences and their corresponding target sequences from the tokenized data, enabling the preparation of data for training a sequence-based model.

### Function: `create_sequences(data, seq_length, stoi)`
#### Inputs:
- **`data`**: The tokenized and flattened text data (a list of words).
- **`seq_length`**: The desired length of each input sequence.
- **`stoi`**: The dictionary mapping words to their unique indices.

#### Outputs:
- **`sequences`**: A list of input sequences, where each sequence is a list of word indices of length `seq_length`.
- **`targets`**: A list of target sequences, also of length `seq_length`, which represent the expected output for each input sequence.

#### Key Steps:
1. **Iterate through the dataset**:
   - The function iterates through the data, extracting a sliding window of size `seq_length` for both inputs (`seq`) and shifted outputs (`target`).
2. **Convert words to indices**:
   - Each word in the sequence and target is converted to its corresponding index using the `stoi` dictionary.
   - If a word is not found in `stoi`, it defaults to index `0`.
3. **Filter sequences by length**:
   - Only sequences and targets that match the desired length (`seq_length`) are added to the output lists.

#### Additional Notes:
- **Progress bar**: The `tqdm` library is used to display the progress of sequence generation, particularly useful for large datasets.
- This function is crucial for preparing the data for models like RNNs or Transformers, which require fixed-length sequences for training.

In [3]:
def create_sequences(data, seq_length, stoi):
    sequences = []
    targets = []
    
    for i in tqdm(range(len(data) - seq_length)):
        # Extract sequence and target
        seq = data[i:i+seq_length]
        target = data[i+1:i+seq_length+1]
        
        # Convert each word in the sequence and target to indices using stoi
        seq_indices = [stoi.get(word, 0) for word in seq]
        target_indices = [stoi.get(word, 0) for word in target]
        
        # Only add sequences and targets of the desired length
        if len(seq_indices) == seq_length and len(target_indices) == seq_length:
            sequences.append(seq_indices)
            targets.append(target_indices)
    
    return sequences, targets

# Load Word2Vec Model and Prepare Dataset
This cell integrates a pre-trained Word2Vec model for word embeddings, prepares training and validation datasets, and sets up PyTorch DataLoaders for efficient batch processing.

### Step 1: Load Pre-trained Word2Vec Model
- The Word2Vec model is loaded from a pre-saved file (`wikitext_small_word2vec.model`).
- Two mappings are created:
  - **`stoi`** (String-to-Index): Maps each word in the Word2Vec vocabulary to its unique index.
  - **`itos`** (Index-to-String): Maps each index back to its corresponding word.

### Step 2: Create Input and Target Sequences
- The `create_sequences` function is used to generate:
  - **`sequences`**: Input sequences from the training data.
  - **`targets`**: Corresponding target sequences.
- Validation sequences (`seq_val`) and targets (`target_val`) are similarly created.
- **Sequence length** is set to `50`, meaning each input and target sequence consists of 50 words.

### Step 3: Define the `TextDataset` Class
- A custom PyTorch dataset class is created to handle sequences and targets:
  - **`__init__`**: Stores the sequences and targets.
  - **`__len__`**: Returns the total number of sequences.
  - **`__getitem__`**: Retrieves a specific sequence and its corresponding target, converting them to PyTorch tensors.

### Step 4: Set Up DataLoaders
- PyTorch DataLoaders are created for both training and validation datasets:
  - **Batch size** is set to `32`.
  - **Shuffling** is enabled for training to ensure randomization during training.
  - Validation DataLoader does not shuffle the data, as it is used for evaluation.

### Purpose:
- This setup enables efficient training and validation of sequence models using batches, with pre-trained word embeddings providing meaningful representations for words.

In [4]:
word2vec = gensim.models.Word2Vec.load('wikitext_small_word2vec.model')
stoi = {word: idx for idx, word in enumerate(word2vec.wv.index_to_key)}
itos = {idx: word for idx, word in enumerate(word2vec.wv.index_to_key)}

# Example usage:
seq_length = 50
sequences, targets = create_sequences(flat_train, seq_length, stoi)
seq_val, target_val = create_sequences(flat_valid, seq_length, stoi)

class TextDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return torch.tensor(self.sequences[idx], dtype=torch.long), torch.tensor(self.targets[idx], dtype=torch.long)

batch_size = 32
dataset = TextDataset(sequences, targets)
dataset_val = TextDataset(seq_val, target_val)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

100%|██████████| 175865/175865 [00:00<00:00, 262313.67it/s]


# Language Model with Pre-trained Word2Vec Embeddings
This class defines a language model using PyTorch, incorporating pre-trained Word2Vec embeddings for word representation and RNN-based architectures for sequence modeling.

### Class: `LanguageModelWord2Vec`
#### Inputs:
- **`vocab_size`**: Size of the vocabulary (number of unique words).
- **`hidden_dim`**: Number of hidden units in the RNN.
- **`num_layers`**: Number of layers in the RNN.
- **`rnn_type`**: Type of RNN to use (`'LSTM'` or `'GRU'`).
- **`word2vec_path`**: Path to the pre-trained Word2Vec model file.

#### Key Components:
1. **Load Word2Vec Embeddings**:
   - A pre-trained Gensim Word2Vec model is loaded.
   - The embedding dimension is inferred from the Word2Vec model.

2. **Embedding Layer Initialization**:
   - The embedding layer is initialized with the pre-trained Word2Vec weights.
   - The weights are frozen (`requires_grad=False`) to prevent updates during training.

3. **Recurrent Neural Network (RNN)**:
   - An RNN (either LSTM or GRU) is initialized.
   - Takes input word embeddings and outputs a sequence of hidden states.

4. **Fully Connected Layer**:
   - A linear layer maps the RNN’s hidden state outputs to vocabulary size, producing logits for each word in the vocabulary.

#### Forward Method:
- **Input**: 
  - `x`: Tensor of word indices with shape `(batch_size, seq_length)`.
  - `hidden`: Initial hidden states for the RNN.
- **Steps**:
  1. Convert word indices to embeddings using the pre-trained embedding layer.
     - Output shape: `(batch_size, seq_length, embedding_dim)`.
  2. Pass embeddings through the RNN.
     - Output shape: `(batch_size, seq_length, hidden_dim)`.
  3. Reshape RNN output and apply the fully connected layer.
     - Final output shape: `(batch_size * seq_length, vocab_size)`.
- **Output**:
  - Predictions (logits) for each word in the vocabulary.
  - Updated hidden states for the RNN.

### Purpose:
This model combines the semantic richness of pre-trained Word2Vec embeddings with the sequential modeling power of RNNs, making it suitable for NLP tasks like next-word prediction or language modeling.

In [5]:
class LanguageModelWord2Vec(nn.Module):
    def __init__(self, vocab_size, hidden_dim, num_layers, rnn_type='LSTM',  word2vec_path='wikitext_small_word2vec.model'):
        super(LanguageModelWord2Vec, self).__init__()
        self.hidden_dim = hidden_dim
        
        # Step 1: Load the Gensim Word2Vec model
        self.word2vec = gensim.models.Word2Vec.load(word2vec_path)
        
        # Get the embedding dimensions from the Gensim model
        embedding_dim = self.word2vec.wv.vector_size

        # Step 2: Initialize the embedding layer with pretrained Word2Vec weights
        
        weights = torch.FloatTensor(self.word2vec.wv.vectors)
        self.embedding = nn.Embedding.from_pretrained(weights)
        
        self.embedding.weight.requires_grad = False
        
        # Step 3: Initialize RNN (LSTM or GRU)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers, batch_first=True)
        else:
            raise ValueError("Invalid RNN type. Choose 'LSTM' or 'GRU'.")
        
        self.fc = nn.Linear(hidden_dim, vocab_size)

    
    def forward(self, x, hidden):
        # x shape: (batch_size, seq_length)
        embeds = self.embedding(x)
        # embeds shape: (batch_size, seq_length, embedding_dim)
        out, hidden = self.rnn(embeds, hidden)
        # out shape: (batch_size, seq_length, hidden_dim)
        out = out.reshape(-1, self.hidden_dim)
        # out shape: (batch_size * seq_length, hidden_dim)
        out = self.fc(out)
        # out shape: (batch_size * seq_length, vocab_size)
        return out, hidden

# Training and Validation of the Language Model
This cell initializes and trains the `LanguageModelWord2Vec` class using pre-trained Word2Vec embeddings. It also evaluates the model on a validation dataset during training to monitor performance.

### Key Steps:
#### 1. **Initialize the Model**
- **Parameters**:
  - `hidden_dim`: Number of hidden units in the RNN.
  - `num_layers`: Number of RNN layers.
  - `rnn_type`: RNN architecture (`'LSTM'` or `'GRU'`).
- The model is initialized with the vocabulary size and moved to the appropriate device (CPU/GPU).

#### 2. **Define Loss Function and Optimizer**
- **Loss Function**: Cross-entropy loss is used to calculate the difference between predicted and actual word indices.
- **Optimizer**: Adam optimizer is used with a learning rate of 0.0003.

#### 3. **Initialize Metrics**
- Word Error Rate (WER) and Perplexity are initialized for both training and validation to evaluate the model:
  - **WER**: Measures the discrepancy between predicted and target word sequences.
  - **Perplexity**: Evaluates the probability of the target sequence under the model's predictions.

#### 4. **Training Loop**
- The training is conducted for a specified number of epochs (`num_epochs`):
  - **Forward Pass**: The model processes input sequences and predicts outputs.
  - **Loss Calculation**: Cross-entropy loss is computed between predictions and targets.
  - **Backpropagation**: Gradients are computed and applied to update the model weights.
  - **Metric Calculation**:
    - WER is updated using predicted and actual sequences.
    - Perplexity is computed using predictions and targets.

#### 5. **Validation Phase**
- After each epoch, the model is evaluated on the validation dataset:
  - Predictions are generated, and validation loss, WER, and perplexity are calculated.
  - The validation metrics are stored for analysis.

#### 6. **Performance Logging**
- For each epoch, the following metrics are printed:
  - Training loss, WER, and perplexity.
  - Validation loss, WER, and perplexity.

### Purpose:
This training loop aims to optimize the language model's performance on the training data while monitoring its generalization to unseen validation data. The use of metrics like WER and perplexity provides insights into the model's accuracy and language fluency.

In [6]:
# Initialize the model
hidden_dim = 256
num_layers = 3
rnn_type = 'LSTM' # Choose 'LSTM' or 'GRU'
model = LanguageModelWord2Vec(vocab_size, hidden_dim, num_layers, rnn_type)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003)

# Initialize WER and Perplexity for both train and validation
wer = WordErrorRate()
val_wer = WordErrorRate()
train_perplexity = Perplexity()
val_perplexity = Perplexity()

if torch.cuda.is_available():
    wer = wer.to(device)
    val_wer = val_wer.to(device)
    train_perplexity = train_perplexity.to(device)
    val_perplexity = val_perplexity.to(device)

train_losses = {'crossentropy': [], 'wer': [], 'perplexity': []}
val_losses = {'crossentropy': [], 'wer': [], 'perplexity': []}

# Move model to device (GPU or CPU)
model.to(device)

# Training loop
num_epochs = 10  
for epoch in range(num_epochs):
    
    model.train()
    total_loss = 0
    total_wer = 0
    hidden = None
    
    # Initialize metrics for the epoch
    wer.reset()
    train_perplexity.reset()
    
    for i, (inputs, targets) in tqdm(enumerate(dataloader), total=len(dataloader)):
        hidden = None
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs, hidden = model(inputs, hidden)
        
        # Detach hidden states to prevent backpropagating through the entire training history
        if isinstance(hidden, tuple):
            hidden = tuple([h.detach() for h in hidden])
        else:
            hidden = hidden.detach()
        
        outputs = outputs.view(-1, vocab_size) 
        targets_buffer = targets
        targets = targets.view(-1)
        
        # Compute loss
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

        # Calculate WER
        # For WER and Perplexity, keep the original targets shape
        preds = torch.argmax(outputs, dim=1)

        spreds = [itos[p.item()] for p in preds]
        stargets = [itos[t.item()] for t in targets]  # You might want to keep the same targets as for loss
        wer.update(spreds, stargets)
        
        # Calculate perplexity using the original (2D) targets
        if torch.cuda.is_available():
            train_perplexity.update(outputs.view(batch_size, seq_length, -1), targets_buffer)
        else:
            train_perplexity.update(outputs.view(batch_size, seq_length, -1).cpu(), targets_buffer.cpu())  # Pass original target shape
        
    # Calculate training metrics
    avg_train_loss = total_loss / len(dataloader)
    avg_train_wer = wer.compute().item()
    avg_train_perplexity = train_perplexity.compute().item()
    
    train_losses['crossentropy'].append(avg_train_loss)
    train_losses['wer'].append(avg_train_wer)
    train_losses['perplexity'].append(avg_train_perplexity)
    

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_total_loss = 0
    val_wer.reset()
    val_perplexity.reset()

    with torch.no_grad():
        for j, val_inputs, val_targets in enumerate(tqdm(dataloader_val, total=len(dataloader_val))):

            # Only look at a part of the validation set
            if j > len(dataloader_val)/3:
                break
            
            val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)

            val_outputs, _ = model(val_inputs, None)

            # Ensure outputs are float32
            val_outputs = val_outputs.float()

            # Reshape outputs and targets for loss calculation
            val_outputs = val_outputs.view(-1, vocab_size)
            val_targets = val_targets.view(-1)

            # Compute validation loss
            val_loss = criterion(val_outputs, val_targets)
            val_total_loss += val_loss.item()

            # Validation WER and Perplexity
            val_preds = torch.argmax(val_outputs, dim=1)
            val_wer.update(val_preds, val_targets)
            if torch.cuda.is_available():
                val_perplexity.update(val_outputs.view(batch_size, seq_length, -1), val_targets)
            else:
                val_perplexity.update(val_outputs.view(batch_size, seq_length, -1).cpu(), val_targets.cpu())

    # Calculate average validation loss, WER, and perplexity
    avg_val_loss = val_total_loss / len(dataloader_val)
    avg_val_wer = val_wer.compute().item()
    avg_val_perplexity = val_perplexity.compute().item()

    val_losses['crossentropy'].append(avg_val_loss)
    val_losses['wer'].append(avg_val_wer)
    val_losses['perplexity'].append(avg_val_perplexity)

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Training WER: {avg_train_wer:.4f}, Training Perplexity: {avg_train_perplexity:.4f}')
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}, Validation WER: {avg_val_wer:.4f}, Validation Perplexity: {avg_val_perplexity:.4f}')


  0%|          | 42/52488 [00:23<8:13:22,  1.77it/s]


KeyboardInterrupt: 

In [6]:
def generate_text(model, seed_word, max_length=50, random_sampling=False):
    model.eval()  # Set the model to evaluation mode
    generated_words = [seed_word]  # List to store generated words
    
    # Convert seed word to index
    seed_idx = torch.tensor([[stoi[seed_word]]]).to(device)  # Shape: (1, 1)

    # Initialize hidden state
    hidden = None
    
    # Loop through to generate words
    for _ in range(max_length):
        # Forward pass through the model
        with torch.no_grad():
            output, hidden = model(seed_idx, hidden)
        
        # Get the predicted word (highest probability or sample)
        output = output.squeeze(1)  # Remove the seq_len dimension (now (1, vocab_size))
        if random_sampling:
            # Sample from the output distribution
            probabilities = torch.softmax(output, dim=1)
            predicted_idx = torch.multinomial(probabilities, num_samples=1).item()
        else:
            predicted_idx = torch.argmax(output, dim=1).item()  # Get the index of the word with highest probability
        
        # Convert index back to word
        predicted_word = itos[predicted_idx]
        
        # Append the predicted word to the list
        generated_words.append(predicted_word)
        
        # Set the predicted word as the next input (shape: (1, 1))
        seed_idx = torch.tensor([[predicted_idx]]).to(device)
        
        # Stop if an end-of-sequence token is generated (optional)
        if predicted_word == "<eos>":  # Assuming "<eos>" is the token for end of sentence
            break
    
    return ' '.join(generated_words)

# Example usage
generated_text = generate_text(model, seed_word="Harry", max_length=50, random_sampling=True)
print(generated_text)


Harry , and - another Thousand inside the dormitory . He was looking very around by a splash , and Lee did not showing him Crookshanks and went about to know I had been in a former relief . “ The skrewts with Household … Had I want to have an


# FastText

This section replicates what has been done with Word2Vec and applies it to FastText.


In [ ]:
fasttext = gensim.models.FastText.load('wikitext_small_fasttext.model')
stoi = {word: idx for idx, word in enumerate(fasttext.wv.index_to_key)}
itos = {idx: word for idx, word in enumerate(fasttext.wv.index_to_key)}

# Example usage:
seq_length = 50
sequences, targets = create_sequences(flat_train, seq_length, stoi)

batch_size = 128
dataset = TextDataset(sequences, targets)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [110]:
class LanguageModelFastText(nn.Module):
    def __init__(self, vocab_size, hidden_dim, num_layers, rnn_type='LSTM',  word2vec_path='wikitext_small_word2vec.model'):
        super(LanguageModelFastText, self).__init__()
        self.hidden_dim = hidden_dim
        
        # Step 1: Load the Gensim Word2Vec model
        self.fasttext = gensim.models.FastText.load(word2vec_path)
        
        # Get the embedding dimensions from the Gensim model
        embedding_dim = self.fasttext.wv.vector_size

        # Step 2: Initialize the embedding layer with pretrained Word2Vec weights
        
        weights = torch.FloatTensor(self.fasttext.wv.vectors)
        self.embedding = nn.Embedding.from_pretrained(weights)
        
        # Freeze embedding layer
        self.embedding.weight.requires_grad = False

        # Step 3: Initialize RNN (LSTM or GRU)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers, batch_first=True)
        else:
            raise ValueError("Invalid RNN type. Choose 'LSTM' or 'GRU'.")
        
        self.fc = nn.Linear(hidden_dim, vocab_size)

    
    def forward(self, x, hidden):
        # x shape: (batch_size, seq_length)
        embeds = self.embedding(x)
        # embeds shape: (batch_size, seq_length, embedding_dim)
        out, hidden = self.rnn(embeds, hidden)
        # out shape: (batch_size, seq_length, hidden_dim)
        out = out.reshape(-1, self.hidden_dim)
        # out shape: (batch_size * seq_length, hidden_dim)
        out = self.fc(out)
        # out shape: (batch_size * seq_length, vocab_size)
        return out, hidden

In [ ]:
# Initialize the model
hidden_dim = 256  

num_layers = 3
rnn_type = 'LSTM' # Choose 'LSTM' or 'GRU'

model = LanguageModelFastText(vocab_size, hidden_dim,  num_layers, rnn_type)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003)

# Move model to device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'mps')

model.to(device)

# Training loop
num_epochs = 3  
for epoch in range(num_epochs):
    
    model.train()
    hidden = None
    total_loss = 0
    
    for inputs, targets in tqdm(dataloader):
        hidden = None
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs, hidden = model(inputs, hidden)
        
        # Detach hidden states to prevent backpropagating through the entire training history
        if isinstance(hidden, tuple):
            hidden = tuple([h.detach() for h in hidden])
        else:
            hidden = hidden.detach()
        
        outputs = outputs.view(-1, vocab_size) 
        targets = targets.view(-1)     
        
        # Compute loss
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    torch.save(model.state_dict(), f'./model_{epoch}.pt')

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
